In [19]:
import sys
sys.path.append("../Scripts/")

import pandas as pd
from pdf_parser import sentence_parser 
import torch
from transformers import AutoTokenizer, T5EncoderModel

# # Load T5EncoderModel - small
# tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small", suppress_warnings=True)
# model = T5EncoderModel.from_pretrained("google-t5/t5-small")

# Load T5EncoderModel - large (>2.8 GB - longer load time)
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-large", suppress_warnings=True)
model = T5EncoderModel.from_pretrained("google-t5/t5-large")

In [14]:
file_path = '../SampleReports/2023_Coles_Report.pdf'

In [15]:
pdf_sentences = sentence_parser(file_path, print_sentences=True ,num_sentences=10, parsing_method='pypdf')

Report located in specified directory

Summary:
Total number of sentences: 686

 2023 Sustainability ReportWorking towards a more sustainable future Coles Group Limited ABN 11 004 089 936Acknowledgement of Country Coles wishes to acknowledge the Traditional Custodians of Country throughout Australia.

 We recognise their strength and resilience and pay our respects to their Elders past and present.

 Coles extends that respect to all Aboriginal and Torres Strait Islander people, and recognises their rich cultures and their continuing connection to land and waters.

 Aboriginal and Torres Strait Islander people are advised that this report may contain names and images of people who are deceased.

 All references to Indigenous and First Nations people in this report are intended to include Aboriginal and/or Torres Strait Islander people.

 Feedback We welcome feedback on this report.

 This report also includes forward-looking statements regarding climate change and other environmental a

In [18]:
# Initialize an empty list to store dictionaries
sentence_data = []

# Iterate over the sentences
for idx, sentence in enumerate(pdf_sentences, start=1):
    # Tokenize Input sentence and query
    input_ids = tokenizer(sentence, return_tensors='pt').input_ids
    query_ids = tokenizer('Sustainable', return_tensors='pt').input_ids

    # Generate Encodings
    outputs = model(input_ids=input_ids)
    query = model(input_ids=query_ids)

    # Retrieve encodings for sentence and query.
    last_hidden_states = outputs.last_hidden_state
    last_hidden_states_query = query.last_hidden_state

    # Apply mean pooling along the token dimension (dim=1)
    pooled_last_hidden_states = torch.mean(last_hidden_states, dim=1).unsqueeze(0)
    pooled_last_hidden_states_query = torch.mean(last_hidden_states_query, dim=1).unsqueeze(0)

    # Reshape pooled_last_hidden_states_query to match the shape of pooled_last_hidden_states
    pooled_last_hidden_states_query = pooled_last_hidden_states_query.view(1, -1)

    # Compute dot product between the pooled representations
    similarity = torch.matmul(pooled_last_hidden_states, pooled_last_hidden_states_query.T)

    # Append a dictionary with score and sentence to the list
    sentence_data.append({'Sentence_ID': idx, 'Score': similarity.item(), 'Sentence': sentence})

# Create DataFrame from the list of dictionaries
df = pd.DataFrame(sentence_data)

NameError: name 'pd' is not defined

In [26]:
# Create DataFrame from the list of dictionaries
df = pd.DataFrame(sentence_data)

# Sort the DataFrame by the "Score" column in descending order
df_sorted = df.sort_values(by='Score', ascending=False)

# Display the sorted DataFrame
(df_sorted.head(100))

,Sentence_ID,Score,Sentence
199,200,3.935318,Coles Collect Our Coles Collect backhaul trans...
27,28,3.801864,"Koi, Daley St)."
510,511,3.685190,Sustain 40/40/201 on the Board. 4.
159,160,3.518558,Category 1 emissions from purchased goods and ...
273,274,3.514151,Overview Energy and emissions Waste Sourcing a...
...,...,...,...
147,148,2.767045,"When building new Coles Supermarkets, the majo..."
56,57,2.761166,Underpinning our strategic pillars are two fou...
563,564,2.752482,"Through this, we work with other organisations..."
641,642,2.748825,Identifying and improving healthier choices Co...
